In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
import numpy as np

/Users/sunnywu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Read the Relevant Datasets

In [2]:
# read the listing data file
listings = gpd.read_file('../data/curated/listings_suburbs_SA2.shp')

In [3]:
# read the shape file for SAL regions
suburbs = gpd.read_file("../data/landing/SAL_2021_AUST_GDA2020_SHP/SAL_2021_AUST_GDA2020.shp")

In [4]:
# read the shape file for SA2 regions
SA2 = gpd.read_file("../data/landing/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")

In [5]:
# select the subset that we care about for the SA2
SA2 = SA2.loc[SA2['STE_NAME21']=='Victoria']
SA2 = SA2[['SA2_CODE21', 'SA2_NAME21', 'geometry']]

In [6]:
# select the subset that we care about for the SAL 
suburbs = suburbs.loc[suburbs['STE_NAME21']=='Victoria']
suburbs = suburbs[['SAL_CODE21', 'SAL_NAME21', 'geometry']]

## plot the median rental price for each SA2 region

In [7]:
# median rent price for each region
rent_price = pd.DataFrame(listings.groupby('SA2_NAME').median()['rent'])

/var/folders/_9/rb9s3yd952d26d6ytbnwjc3h0000gn/T/ipykernel_32536/255880823.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rent_price = pd.DataFrame(listings.groupby('SA2_NAME').median()['rent'])


In [8]:
# take the log of rent price for a better plotting
rent_price['rent'] = np.log(rent_price['rent'])

In [9]:
# set the index
rent_price['SA2'] = rent_price.index
rent_price.index = range(rent_price.shape[0])

In [10]:
rent_price_merged = SA2.merge(rent_price, left_on = "SA2_NAME21", right_on = "SA2", how ='inner')

In [11]:
# create a JSON object about location and their respective geomtry
geoJSON = SA2[['SA2_NAME21', 'geometry']].to_json()

In [12]:
# plot the price map for residence
m1 = folium.Map(location=[-37.8136, 144.9631], tiles="Stamen Terrain", zoom_start=10)

folium.Choropleth(
    geo_data=geoJSON,
    data=rent_price,
    columns = ['SA2','rent'],
    key_on='properties.SA2_NAME21',  
    fill_color='YlOrRd', 
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='log of median rent price',
).add_to(m1)

m1.save('../plots/rent_price_SA2.html')

In [13]:
# fill the studios that does not have a room feature to default 1
listings['rooms'].fillna(1, inplace=True)

In [14]:
# make a new column that calculate the mean rent price per room
listings['per_room'] = np.log(listings['rent']/listings['rooms'])

In [15]:
# median rent price for each region
rent_price = pd.DataFrame(listings.groupby('SA2_NAME').median()['per_room'])

/var/folders/_9/rb9s3yd952d26d6ytbnwjc3h0000gn/T/ipykernel_32536/4250100687.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rent_price = pd.DataFrame(listings.groupby('SA2_NAME').median()['per_room'])


In [16]:
# set the index
rent_price['SA2'] = rent_price.index
rent_price.index = range(rent_price.shape[0])

In [17]:
rent_price_merged = SA2.merge(rent_price, left_on = "SA2_NAME21", right_on = "SA2", how ='inner')

In [18]:
# plot the price map for residence
m2 = folium.Map(location=[-37.8136, 144.9631], tiles="Stamen Terrain", zoom_start=10)

folium.Choropleth(
    geo_data=geoJSON,
    data=rent_price,
    columns = ['SA2','per_room'],
    key_on='properties.SA2_NAME21',  
    fill_color='YlOrRd', 
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='log of median rent per room price',
).add_to(m2)

m2.save('../plots/rent_price_pr_SA2.html')